In [1]:
from findata_tools import *

In [2]:
import numpy as np
import pandas as pd

In [3]:
%pwd

'/Users/yg/code/github/DS-GA-1013/hw/hw2/findata'

In [4]:
names, days = load_data('stockprices.csv')
(n_days, n_stocks) = days.shape
print(days.shape)
n_returns = n_days - 1
returns = days[1:] - days[:-1]
print(returns.shape, returns)

(433, 18)
(432, 18) [[ -2.550804  -3.200012   0.239204 ...   0.26794   -0.239998  -0.32    ]
 [ -1.942085  -1.139954  -0.956821 ...  -1.63744    2.079994  -0.55    ]
 [ -4.106422 -24.710022  -1.798817 ...  -2.828319   4.040001  -0.21    ]
 ...
 [ -1.210007 -12.599976  -0.149994 ...   0.369995  -0.159999  -0.03    ]
 [  0.059998  -4.330017   0.279998 ...   0.139999  -0.090001  -0.06    ]
 [ -2.659989   3.350037  -0.5      ...   0.080002  -0.039997   0.13    ]]


/Users/yg/code/github/DS-GA-1013/hw/hw2/findata/findata_tools.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = df.as_matrix()


In [5]:
centered_returns = returns - np.mean(returns, axis=0)
cov = np.cov(centered_returns,rowvar=False)
U, S, principal_directions = np.linalg.svd(cov)
max_coeff_stocks_index = [np.argmax(np.abs(principal_directions[k])) for k in range(2)]
max_coeff_stocks = [names[k] for k in max_coeff_stocks_index]

print(max_coeff_stocks)
pretty_print(np.sum(np.abs(returns),axis = 0),names)

['amzn', 'goog']
       aapl         amzn        msft         goog         xom         apc  \
  454.14342  3423.779352  208.844523  2799.940424  258.501684  388.899131   

         cvx           c          gs       jpm         aet         jnj  \
  381.849154  247.140956  901.311051  277.9143  505.556593  275.823333   

         dgx         spy        xlf         sso         sds    uso  
  257.409551  424.583189  64.368439  277.841774  276.939997  77.16  


In [6]:
print(names)

['aapl', 'amzn', 'msft', 'goog', 'xom', 'apc', 'cvx', 'c', 'gs', 'jpm', 'aet', 'jnj', 'dgx', 'spy', 'xlf', 'sso', 'sds', 'uso']


In [7]:
df = pd.DataFrame(list(np.sum(np.abs(returns),axis = 0)))
df = df.rename(columns={0: "return"})
df["stock"] = names
df.sort_values("return", inplace=True)
df

,return,stock
14,64.368439,xlf
17,77.160000,uso
2,208.844523,msft
7,247.140956,c
12,257.409551,dgx
4,258.501684,xom
11,275.823333,jnj
16,276.939997,sds
15,277.841774,sso
9,277.914300,jpm


In [8]:
print("{}, {}, {}, {}"
      .format(3423.779352/901.311051, 3423.779352/64.368439 , 2799.940424 / 901.311051, 2799.940424 /64.368439 ))

3.798665675075585, 53.19034304995341, 3.106519576003734, 43.49865349383415


In [9]:
cov2 = np.matmul(centered_returns.T, centered_returns)/431

In [10]:
print(np.allclose(cov, cov2))
cov[0,:]

True


array([ 2.32677149,  7.25243643,  0.48646541,  6.53157405,  0.21110301,
        0.25967979,  0.30900248,  0.37293323,  1.27976813,  0.41644247,
        0.60466426,  0.24751404,  0.40789532,  1.15290292,  0.07012189,
        0.75382492, -0.79060188,  0.04372071])

In [11]:
cov2[0,:]

array([ 2.32677149,  7.25243643,  0.48646541,  6.53157405,  0.21110301,
        0.25967979,  0.30900248,  0.37293323,  1.27976813,  0.41644247,
        0.60466426,  0.24751404,  0.40789532,  1.15290292,  0.07012189,
        0.75382492, -0.79060188,  0.04372071])

In [12]:
u, s, v = np.linalg.svd(centered_returns)

In [13]:
w, v2 = np.linalg.eigh(cov2)

In [14]:
w

array([1.40417957e-03, 2.33558560e-02, 4.57528978e-02, 6.67684969e-02,
       8.75528773e-02, 1.56047940e-01, 2.29071780e-01, 2.48852621e-01,
       3.95630157e-01, 5.02951878e-01, 6.45762928e-01, 8.77622561e-01,
       1.58665419e+00, 2.08250145e+00, 2.48052451e+00, 1.09623482e+01,
       3.83867869e+01, 1.69479899e+02])

In [15]:
np.power(s,2)/431

array([1.69479899e+02, 3.83867869e+01, 1.09623482e+01, 2.48052451e+00,
       2.08250145e+00, 1.58665419e+00, 8.77622561e-01, 6.45762928e-01,
       5.02951878e-01, 3.95630157e-01, 2.48852621e-01, 2.29071780e-01,
       1.56047940e-01, 8.75528773e-02, 6.67684969e-02, 4.57528978e-02,
       2.33558560e-02, 1.40417957e-03])

In [16]:
print(np.allclose(w[:2], (np.power(s,2)/431)[:2]))

False


In [17]:
np.allclose(v.T[:,0], v2[:,0])
np.allclose(v.T[:,1], v2[:,1])

False

In [18]:
max_coeff_stocks_index = [np.argmax(np.abs(v2[:, k])) for k in range(2)]
max_coeff_stocks = [names[k] for k in max_coeff_stocks_index]

print(max_coeff_stocks)
pretty_print(np.sum(np.abs(returns),axis = 0),names)

['sso', 'uso']
       aapl         amzn        msft         goog         xom         apc  \
  454.14342  3423.779352  208.844523  2799.940424  258.501684  388.899131   

         cvx           c          gs       jpm         aet         jnj  \
  381.849154  247.140956  901.311051  277.9143  505.556593  275.823333   

         dgx         spy        xlf         sso         sds    uso  
  257.409551  424.583189  64.368439  277.841774  276.939997  77.16  


# 3 - B Question

In [19]:
normed_returns = centered_returns / returns.std(axis=0)

In [20]:
normed_returns.std(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

In [21]:
cov3 = np.matmul(normed_returns.T, normed_returns)/431
w, v2 = np.linalg.eig(cov3)

In [32]:
# for k in range(2):
#     print("PD")
#     pretty_print(v2[:, k],names)

In [24]:
U, S, principal_directions_stand = np.linalg.svd(np.corrcoef(centered_returns,rowvar=False))
for k in range(2):
    print("PD")
    pretty_print(principal_directions_stand[k],names)

PD
      aapl      amzn      msft      goog       xom       apc       cvx  \
 -0.184648 -0.168095 -0.227645 -0.197887 -0.208861 -0.194808 -0.223865   

         c        gs      jpm       aet       jnj       dgx       spy  \
 -0.280147 -0.260443 -0.27294 -0.188555 -0.150783 -0.196612 -0.336632   

       xlf       sso       sds       uso  
 -0.198364 -0.334826  0.327259 -0.159213  
PD
      aapl    amzn      msft      goog       xom       apc       cvx  \
  0.285821  0.3302  0.302005  0.389346 -0.293444 -0.329116 -0.308272   

         c        gs       jpm       aet       jnj       dgx       spy  \
 -0.133288 -0.137664 -0.155599  0.071286  0.108239  0.165412  0.084872   

       xlf       sso       sds       uso  
 -0.124933  0.091568 -0.056002 -0.370943  


In [156]:
def extract_stock_price(index_day, prices, index_stock, names):
    stock_name = names[index_stock]
    daily_price = prices[index_day,:]
    stock_price = daily_price[index_stock]
    return stock_price, [stock_name]

def extract_stock_weights(index_stock_end, principal_direction, principal_direction_index, 
                          index_day, prices, names, index_stock_start=None):
    if index_stock_start:
        weigth_stocks = principal_direction[index_stock_start:index_stock_end, principal_direction_index]
        stock_names = names[index_stock_start:index_stock_end]
        daily_prices = prices[index_day,:]
        stock_prices = daily_prices[index_stock_start:index_stock_end]
    else:
        weigth_stocks = principal_direction[:index_stock, principal_direction_index]
        stock_names = names[:index_stock]
        daily_prices = prices[index_day,:]
        stock_prices = daily_prices[:index_stock]
    return daily_prices, weigth_stocks, stock_prices, stock_names

def compute_weighted_average(weigth_stocks, price_stocks):
    return np.sum(np.multiply(weigth_stocks, price_stocks), axis=0)/ np.sum(weigth_stocks, axis=0)

# SPY

State Street's SPDR S&P 500 ETF.  A security that roughly tracks the S&P 500, a weighted average of the stock prices of 500 top US companies.

In [145]:
index_day = 0
index_stock_end = 13
principal_direction_index = 0
daily_prices, weigth_stocks, stock_prices, stock_names = extract_stock_weights(index_stock_end, v2, principal_direction_index,
                                                                                index_day, days, names)
print(80*"-")
print("Weights coefficients for the first 13 stock prices using PD{}".format(principal_direction_index))
pretty_print(weigth_stocks, stock_names)
print(80*"-")
print("Day{} prices".format(index_day))
pretty_print(stock_prices, stock_names)
print(80*"-")
print("Day{}, computed SPY weighted average prices:{}".format(index_day, compute_weighted_average(weigth_stocks, stock_prices)))
print(80*"-")
print("Day{} SPY price".format(index_day))
spy_return, spy_name = extract_stock_price(index_day, days, index_stock_end, names)
pretty_print(spy_return, spy_name)

--------------------------------------------------------------------------------
Weights coefficients for the first 13 stock prices using PD0
      aapl      amzn      msft      goog       xom       apc       cvx  \
 -0.184648 -0.168095 -0.227645 -0.197887 -0.208861 -0.194808 -0.223865   

         c        gs      jpm       aet       jnj       dgx       spy  \
 -0.280147 -0.260443 -0.27294 -0.188555 -0.150783 -0.196612 -0.336632   

       xlf       sso       sds  
 -0.198364 -0.334826  0.327259  
--------------------------------------------------------------------------------
Day0 prices
        aapl       amzn       msft        goog        xom        apc  \
  101.790649  636.98999  52.433533  741.840027  72.740799  48.801582   

        cvx          c          gs       jpm         aet        jnj  \
  82.577927  50.149845  172.800156  61.18005  107.291946  95.826782   

        dgx         spy        xlf        sso   sds  
  68.369064  194.027725  14.100877  60.724442  82.0  
-------

In [146]:
index_day = 100
index_stock_end = 13
principal_direction_index = 0
daily_prices, weigth_stocks, stock_prices, stock_names = extract_stock_weights(index_stock_end, v2, principal_direction_index,
                                                                                index_day, days, names)
print(80*"-")
print("Weights coefficients for the first 13 stock prices using PD{}".format(principal_direction_index))
pretty_print(weigth_stocks, stock_names)
print(80*"-")
print("Day{} prices".format(index_day))
pretty_print(stock_prices, stock_names)
print(80*"-")
print("Day{}, computed SPY weighted average prices:{}".format(index_day, compute_weighted_average(weigth_stocks, stock_prices)))
print(80*"-")
print("Day{} SPY price".format(index_day))
spy_return, spy_name = extract_stock_price(index_day, days, index_stock_end, names)
pretty_print(spy_return, spy_name)

--------------------------------------------------------------------------------
Weights coefficients for the first 13 stock prices using PD0
      aapl      amzn      msft      goog       xom       apc       cvx  \
 -0.184648 -0.168095 -0.227645 -0.197887 -0.208861 -0.194808 -0.223865   

         c        gs      jpm       aet       jnj       dgx       spy  \
 -0.280147 -0.260443 -0.27294 -0.188555 -0.150783 -0.196612 -0.336632   

       xlf       sso       sds  
 -0.198364 -0.334826  0.327259  
--------------------------------------------------------------------------------
Day100 prices
       aapl        amzn       msft        goog        xom        apc  \
  98.137901  714.909973  50.355423  724.119995  85.809525  51.987953   

        cvx          c          gs        jpm         aet         jnj  \
  96.541328  45.329781  155.994034  62.998978  111.555122  109.226227   

        dgx         spy        xlf        sso        sds  
  75.464516  203.100464  14.347868  65.536652  72.

In [147]:
index_day = 0
index_stock_end = 13
principal_direction_index = 1
daily_prices, weigth_stocks, stock_prices, stock_names = extract_stock_weights(index_stock_end, v2, principal_direction_index,
                                                                                index_day, days, names)
print(80*"-")
print("Weights coefficients for the first 13 stock prices using PD{}".format(principal_direction_index))
pretty_print(weigth_stocks, stock_names)
print(80*"-")
print("Day{} prices".format(index_day))
pretty_print(stock_prices, stock_names)
print(80*"-")
print("Day{}, computed SPY weighted average prices:{}".format(index_day, compute_weighted_average(weigth_stocks, stock_prices)))
print(80*"-")
print("Day{} SPY price".format(index_day))
spy_return, spy_name = extract_stock_price(index_day, days, index_stock_end, names)
pretty_print(spy_return, spy_name)

--------------------------------------------------------------------------------
Weights coefficients for the first 13 stock prices using PD1
      aapl    amzn      msft      goog       xom       apc       cvx  \
  0.285821  0.3302  0.302005  0.389346 -0.293444 -0.329116 -0.308272   

         c        gs       jpm       aet       jnj       dgx       spy  \
 -0.133288 -0.137664 -0.155599  0.071286  0.108239  0.165412  0.084872   

       xlf       sso       sds  
 -0.124933  0.091568 -0.056002  
--------------------------------------------------------------------------------
Day0 prices
        aapl       amzn       msft        goog        xom        apc  \
  101.790649  636.98999  52.433533  741.840027  72.740799  48.801582   

        cvx          c          gs       jpm         aet        jnj  \
  82.577927  50.149845  172.800156  61.18005  107.291946  95.826782   

        dgx         spy        xlf        sso   sds  
  68.369064  194.027725  14.100877  60.724442  82.0  
---------

In [148]:
index_day = 100
index_stock_end = 13
principal_direction_index = 1
daily_prices, weigth_stocks, stock_prices, stock_names = extract_stock_weights(index_stock_end, v2, principal_direction_index,
                                                                                index_day, days, names)
print(80*"-")
print("Weights coefficients for the first 13 stock prices using PD{}".format(principal_direction_index))
pretty_print(weigth_stocks, stock_names)
print(80*"-")
print("Day{} prices".format(index_day))
pretty_print(stock_prices, stock_names)
print(80*"-")
print("Day{}, computed SPY weighted average prices:{}".format(index_day, compute_weighted_average(weigth_stocks, stock_prices)))
print(80*"-")
print("Day{} SPY price".format(index_day))
spy_return, spy_name = extract_stock_price(index_day, days, index_stock_end, names)
pretty_print(spy_return, spy_name)

--------------------------------------------------------------------------------
Weights coefficients for the first 13 stock prices using PD1
      aapl    amzn      msft      goog       xom       apc       cvx  \
  0.285821  0.3302  0.302005  0.389346 -0.293444 -0.329116 -0.308272   

         c        gs       jpm       aet       jnj       dgx       spy  \
 -0.133288 -0.137664 -0.155599  0.071286  0.108239  0.165412  0.084872   

       xlf       sso       sds  
 -0.124933  0.091568 -0.056002  
--------------------------------------------------------------------------------
Day100 prices
       aapl        amzn       msft        goog        xom        apc  \
  98.137901  714.909973  50.355423  724.119995  85.809525  51.987953   

        cvx          c          gs        jpm         aet         jnj  \
  96.541328  45.329781  155.994034  62.998978  111.555122  109.226227   

        dgx         spy        xlf        sso        sds  
  75.464516  203.100464  14.347868  65.536652  72.55

# XLF: State Street's SPDR Financials ETF.  A security that tracks a weighted average of top US financial companies.

In [149]:
for k in range(2):
    print("{} PD".format(k+1))
    pretty_print(principal_directions_stand[k],names)

1 PD
      aapl      amzn      msft      goog       xom       apc       cvx  \
 -0.184648 -0.168095 -0.227645 -0.197887 -0.208861 -0.194808 -0.223865   

         c        gs      jpm       aet       jnj       dgx       spy  \
 -0.280147 -0.260443 -0.27294 -0.188555 -0.150783 -0.196612 -0.336632   

       xlf       sso       sds       uso  
 -0.198364 -0.334826  0.327259 -0.159213  
2 PD
      aapl    amzn      msft      goog       xom       apc       cvx  \
  0.285821  0.3302  0.302005  0.389346 -0.293444 -0.329116 -0.308272   

         c        gs       jpm       aet       jnj       dgx       spy  \
 -0.133288 -0.137664 -0.155599  0.071286  0.108239  0.165412  0.084872   

       xlf       sso       sds       uso  
 -0.124933  0.091568 -0.056002 -0.370943  


In [160]:
index_day = 100
index_stock = 14
index_stocks_start = 7
index_stocks_end = 10
principal_direction_index = 0
daily_prices, weigth_stocks, stock_prices, stock_names = extract_stock_weights(index_stocks_end, v2, principal_direction_index,
                                                                                index_day, days, names, index_stocks_start)
print(80*"-")
print("Weights coefficients for the first 3 financial stock prices using PD{}".format(principal_direction_index))
pretty_print(weigth_stocks, stock_names)
print(80*"-")
print("Day{} prices".format(index_day))
pretty_print(stock_prices, stock_names)
print(80*"-")
print("Day{}, computed XLF weighted average price:{}".format(index_day, compute_weighted_average(weigth_stocks, stock_prices)))
print(80*"-")
print("Day{} XLF return".format(index_day))
stock_price, stock_name = extract_stock_price(index_day, days, index_stock, names)
pretty_print(stock_price, stock_name)

--------------------------------------------------------------------------------
Weights coefficients for the first 3 financial stock prices using PD0
         c        gs      jpm
 -0.280147 -0.260443 -0.27294
--------------------------------------------------------------------------------
Day100 prices
          c          gs        jpm
  45.329781  155.994034  62.998978
--------------------------------------------------------------------------------
Day100, computed XLF weighted average price:86.68575537909774
--------------------------------------------------------------------------------
Day100 XLF return
        xlf
  14.347868


In [118]:
index_day = 100
index_stocks_start = 7
index_stocks_end = 10
principal_direction_index = 0
daily_return, weigth_stocks, stock_returns, stock_names = extract_stock_weights(index_stocks_end, v2, principal_direction_index,
                                                                                index_day, days, names, index_stocks_start)
print(80*"-")
print("Weights coefficients for the first 13 stock prices using PD{}".format(principal_direction_index))
pretty_print(weigth_stocks, stock_names)
print(80*"-")
print("Day{} returns".format(index_day))
pretty_print(stock_returns, stock_names)
print(80*"-")
print("Day{}, computed XLF weighted average:{}".format(index_day, compute_weighted_average(weigth_stocks, stock_returns)))
print(80*"-")
print("Day{} XLF return".format(index_day))
spy_return, spy_name = extract_stock_return(index_day, days, index_stock, names)
pretty_print(spy_return, spy_name)

--------------------------------------------------------------------------------
Weights coefficients for the first 13 stock prices using PD0
         c        gs      jpm
 -0.280147 -0.260443 -0.27294
--------------------------------------------------------------------------------
Day100 returns
          c          gs        jpm
  45.329781  155.994034  62.998978
--------------------------------------------------------------------------------
Day100, computed XLF weighted average:86.68575537909774
--------------------------------------------------------------------------------
Day100 XLF return
        xlf
  14.347868


In [161]:
index_day = 0
index_stock = 14
index_stocks_start = 7
index_stocks_end = 10
principal_direction_index = 1
daily_prices, weigth_stocks, stock_prices, stock_names = extract_stock_weights(index_stocks_end, v2, principal_direction_index,
                                                                                index_day, days, names, index_stocks_start)
print(80*"-")
print("Weights coefficients for the first 3 financial stock prices using PD{}".format(principal_direction_index))
pretty_print(weigth_stocks, stock_names)
print(80*"-")
print("Day{} prices".format(index_day))
pretty_print(stock_prices, stock_names)
print(80*"-")
print("Day{}, computed XLF weighted average price:{}".format(index_day, compute_weighted_average(weigth_stocks, stock_prices)))
print(80*"-")
print("Day{} XLF return".format(index_day))
stock_price, stock_name = extract_stock_price(index_day, days, index_stock, names)
pretty_print(stock_price, stock_name)

--------------------------------------------------------------------------------
Weights coefficients for the first 3 financial stock prices using PD1
         c        gs       jpm
 -0.133288 -0.137664 -0.155599
--------------------------------------------------------------------------------
Day0 prices
          c          gs       jpm
  50.149845  172.800156  61.18005
--------------------------------------------------------------------------------
Day0, computed XLF weighted average price:93.75722485348642
--------------------------------------------------------------------------------
Day0 XLF return
        xlf
  14.100877


In [162]:
index_day = 100
index_stock = 14
index_stocks_start = 7
index_stocks_end = 10
principal_direction_index = 1
daily_prices, weigth_stocks, stock_prices, stock_names = extract_stock_weights(index_stocks_end, v2, principal_direction_index,
                                                                                index_day, days, names, index_stocks_start)
print(80*"-")
print("Weights coefficients for the first 3 financial stock prices using PD{}".format(principal_direction_index))
pretty_print(weigth_stocks, stock_names)
print(80*"-")
print("Day{} prices".format(index_day))
pretty_print(stock_prices, stock_names)
print(80*"-")
print("Day{}, computed XLF weighted average price:{}".format(index_day, compute_weighted_average(weigth_stocks, stock_prices)))
print(80*"-")
print("Day{} XLF return".format(index_day))
stock_price, stock_name = extract_stock_price(index_day, days, index_stock, names)
pretty_print(stock_price, stock_name)

--------------------------------------------------------------------------------
Weights coefficients for the first 3 financial stock prices using PD1
         c        gs       jpm
 -0.133288 -0.137664 -0.155599
--------------------------------------------------------------------------------
Day100 prices
          c          gs        jpm
  45.329781  155.994034  62.998978
--------------------------------------------------------------------------------
Day100, computed XLF weighted average price:87.49062672340933
--------------------------------------------------------------------------------
Day100 XLF return
        xlf
  14.347868


# SSO ProShares levered ETF that roughly corresponds to twice the daily performance of the S&P 500.

In [125]:
index_day=200
index_stock = -1
daily_return = returns[index_day,:]
stock_returns = daily_return[:]
pretty_print(stock_returns, names)

      aapl      amzn      msft     goog       xom       apc       cvx  \
 -0.343941  0.039978 -0.126941  6.23999  0.385491  0.956391  0.461364   

         c        gs       jpm       aet       jnj      dgx       spy  \
  0.483455  1.856705  0.639229 -0.366638 -0.798614 -0.71032  0.558686   

       xlf      sso       sds   uso  
  0.167292  0.33892 -0.240006  0.21  


In [123]:
print("Rough correlation of SPY with the mean of the first 12 stocks among 18 stocks") 
print("First PD, SPY:{}, rough average:{}".format(v2[13, 0], v2[:13, 0].mean()))
print("Second PD, SPY:{}, rough average:{}".format(v2[13, 1], v2[:13,1].mean()))

Rough correlation of SPY with the mean of the first 12 stocks among 18 stocks
First PD, SPY:-0.3366318246190488, rough average:-0.21194525678931014
Second PD, SPY:0.08487248667261116, rough average:0.022686615723236207


In [125]:
print("Rough correlation of XLF with the mean of the US financial companies among 18 stocks") 
print("First PD, XLF:{}, rough average:{}".format(v2[14, 0], v2[7:10, 0].mean()))
print("Second PD, XLF:{}, rough average:{}".format(v2[14,1], v2[7:10, 1].mean()))

Rough correlation of XLF with the mean of the US financial companies among 18 stocks
First PD, XLF:-0.19836417024240857, rough average:-0.27117654947312425
Second PD, XLF:-0.12493297939512506, rough average:-0.14218371263817778


# SSO ProShares levered ETF that roughly corresponds to twice the daily performance of the S&P 500.

In [ ]:
ProShares levered ETF that roughly corresponds to twice
    # the daily performance of the S&P 500.

In [163]:
print("SSO roughly twice daily performance of SPY") 
print("First PD, SSO:{}, 2 x SPY:{}".format(v2[15, 0], v2[13, 0]))
print("Second PD, SSO:{}, 2 x SPY:{}".format(v2[15, 0], v2[13, 1]))

SSO roughly twice daily performance of SPY
First PD, SSO:-0.33482642597223305, 2 x SPY:-0.3366318246190488
Second PD, SSO:-0.33482642597223305, 2 x SPY:0.08487248667261116


In [129]:
for k in range(2):
    print("{} PD".format(k+1))
    pretty_print(principal_directions_stand[k],names)

1 PD
      aapl      amzn      msft      goog       xom       apc       cvx  \
 -0.184648 -0.168095 -0.227645 -0.197887 -0.208861 -0.194808 -0.223865   

         c        gs      jpm       aet       jnj       dgx       spy  \
 -0.280147 -0.260443 -0.27294 -0.188555 -0.150783 -0.196612 -0.336632   

       xlf       sso       sds       uso  
 -0.198364 -0.334826  0.327259 -0.159213  
2 PD
      aapl    amzn      msft      goog       xom       apc       cvx  \
  0.285821  0.3302  0.302005  0.389346 -0.293444 -0.329116 -0.308272   

         c        gs       jpm       aet       jnj       dgx       spy  \
 -0.133288 -0.137664 -0.155599  0.071286  0.108239  0.165412  0.084872   

       xlf       sso       sds       uso  
 -0.124933  0.091568 -0.056002 -0.370943  


# SDS ProShares inverse levered ETF that roughly corresponds to 
    twice the negative daily performance of the S&P 500.  That is,
    when the S&P 500 goes up by a dollar, this roughly goes down by 2.

In [130]:
for k in range(2):
    print("{} PD".format(k+1))
    pretty_print(principal_directions_stand[k],names)

1 PD
      aapl      amzn      msft      goog       xom       apc       cvx  \
 -0.184648 -0.168095 -0.227645 -0.197887 -0.208861 -0.194808 -0.223865   

         c        gs      jpm       aet       jnj       dgx       spy  \
 -0.280147 -0.260443 -0.27294 -0.188555 -0.150783 -0.196612 -0.336632   

       xlf       sso       sds       uso  
 -0.198364 -0.334826  0.327259 -0.159213  
2 PD
      aapl    amzn      msft      goog       xom       apc       cvx  \
  0.285821  0.3302  0.302005  0.389346 -0.293444 -0.329116 -0.308272   

         c        gs       jpm       aet       jnj       dgx       spy  \
 -0.133288 -0.137664 -0.155599  0.071286  0.108239  0.165412  0.084872   

       xlf       sso       sds       uso  
 -0.124933  0.091568 -0.056002 -0.370943  


In [128]:
print("SDS roughly twice daily the negative daily performance of SPY") 
print("First PD, SSO:{}, 2 x SPY:{}".format(v2[15, 0], 2 * v2[13, 0]))
print("Second PD, SSO:{}, 2 x SPY:{}".format(v2[15, 1], 2 * v2[13, 1]))

SDS roughly twice daily the negative daily performance of SPY
First PD, SSO:-0.33482642597223305, 2 x SPY:-0.6732636492380976
Second PD, SSO:0.09156766289103946, 2 x SPY:0.16974497334522232


# USO Exchange traded product that tracks the price of oil in the US.

In [127]:
index_day=200
index_stock = -1
daily_return = returns[index_day,:]
stock_returns = daily_return[:]
pretty_print(stock_returns, names)

      aapl      amzn      msft     goog       xom       apc       cvx  \
 -0.343941  0.039978 -0.126941  6.23999  0.385491  0.956391  0.461364   

         c        gs       jpm       aet       jnj      dgx       spy  \
  0.483455  1.856705  0.639229 -0.366638 -0.798614 -0.71032  0.558686   

       xlf      sso       sds   uso  
  0.167292  0.33892 -0.240006  0.21  


In [128]:
for k in range(2):
    print("{} PD".format(k+1))
    pretty_print(principal_directions_stand[k],names)

1 PD
      aapl      amzn      msft      goog       xom       apc       cvx  \
 -0.184648 -0.168095 -0.227645 -0.197887 -0.208861 -0.194808 -0.223865   

         c        gs      jpm       aet       jnj       dgx       spy  \
 -0.280147 -0.260443 -0.27294 -0.188555 -0.150783 -0.196612 -0.336632   

       xlf       sso       sds       uso  
 -0.198364 -0.334826  0.327259 -0.159213  
2 PD
      aapl    amzn      msft      goog       xom       apc       cvx  \
  0.285821  0.3302  0.302005  0.389346 -0.293444 -0.329116 -0.308272   

         c        gs       jpm       aet       jnj       dgx       spy  \
 -0.133288 -0.137664 -0.155599  0.071286  0.108239  0.165412  0.084872   

       xlf       sso       sds       uso  
 -0.124933  0.091568 -0.056002 -0.370943  


In [130]:
print("Rough correlation of USO that tracks price of US oil with the mean of the US oil companies among 18 stocks") 
print("First PD, USO:{}, rough average:{}".format(v2[-1, 0], v2[4:7, 0].mean()))
print("Second PD, USO:{}, rough average:{}".format(v2[-1, 1], v2[4:7, 1].mean()))

Rough correlation of USO that tracks price of US oil with the mean of the US oil companies among 18 stocks
First PD, USO:-0.1592126731469417, rough average:-0.2091779373188841
Second PD, USO:-0.3709433376773427, rough average:-0.3102774984100604


# Q3 - C

In [154]:
cov2.shape

(18, 18)

In [164]:
shares = [100 for _ in range(n_stocks)]
double_shares_stocks = ["aapl","amzn","msft","goog"]
shares = np.array([2 * shares[i] if names[i] in double_shares_stocks else shares[i] for i in range(n_stocks)]).reshape(-1,1)
np.sqrt(np.dot(shares.T, np.matmul(cov, shares))).item()

4309.949525975801

# Q4 - d

In [166]:
daily_portfolio_returns = np.dot(returns, shares) 
p = sum([daily_portfolio_returns[i] <= -1000 for i in range(n_returns)])/n_returns
print(p.item())

0.3425925925925926


In [167]:
daily_portfolio_returns.shape

(432, 1)

In [168]:
n_returns

432

In [169]:
np.std(daily_portfolio_returns)

4304.958268336103